In [7]:
import argparse
import h5py
import numpy as np
import plotly.graph_objects as go
import os
from utils.analysis import compute_fft, compute_spectrogram


def load_waveform(file, sweep, channel):
    with h5py.File(file, 'r') as h5f:
        group = h5f[f'sweep_{sweep:03d}'][channel]
        t = group['time'][:]
        v = group['voltage'][:]
        attrs = dict(group.attrs)
    return t, v, attrs


def plot_time_series(t, v, channel):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=t, y=v, mode='lines', name=f"{channel}"))
    fig.update_layout(title=f"Waveform: {channel}", xaxis_title="Time (s)", yaxis_title="Voltage (V)")
    fig.show()


def plot_fft(f, mag, channel):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=f, y=mag, mode='lines', name=f"FFT {channel}"))
    fig.update_layout(title=f"FFT Magnitude: {channel}", xaxis_title="Frequency (Hz)", yaxis_title="Amplitude")
    fig.show()


def plot_spectrogram(Sxx, f, t_spec):
    fig = go.Figure(data=go.Heatmap(z=10 * np.log10(Sxx), x=t_spec, y=f, colorscale='Viridis'))
    fig.update_layout(title="Spectrogram", xaxis_title="Time (s)", yaxis_title="Frequency (Hz)")
    fig.show()


def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("h5file", help="Path to HDF5 file")
    parser.add_argument("--channel", default="CHAN1", help="Channel to plot")
    parser.add_argument("--sweep", type=int, default=0, help="Sweep number (e.g. 0 for sweep_000)")
    parser.add_argument("--fft", action="store_true", help="Show FFT plot")
    parser.add_argument("--spectrogram", action="store_true", help="Show spectrogram")
    args = parser.parse_args()

    t, v, attrs = load_waveform(args.h5file, args.sweep, args.channel)
    plot_time_series(t, v, args.channel)

    fs = attrs.get('sample_rate', 1.0 / (t[1] - t[0]))

    if args.fft:
        f, mag = compute_fft(v, fs)
        plot_fft(f, mag, args.channel)

    if args.spectrogram:
        f_spec, t_spec, Sxx = compute_spectrogram(v, fs)
        plot_spectrogram(Sxx, f_spec, t_spec)




In [20]:
h5file = 'data/data_4.h5'
sweep = 0
channel = 'CHAN2'

t, v, attrs = load_waveform(h5file, sweep, channel)
plot_time_series(t, v, channel)

In [21]:
fs = attrs.get('sample_rate', 1.0 / (t[1] - t[0]))

f, mag = compute_fft(v, fs)
plot_fft(f, mag, channel)

In [22]:
f_spec, t_spec, Sxx = compute_spectrogram(v, fs)
plot_spectrogram(Sxx, f_spec, t_spec)

In [18]:
fs

np.float64(50000000.0)

In [23]:
import pyvisa
rm = pyvisa.ResourceManager()
#rm.list_resources()

mso = rm.open_resource('TCPIP::169.254.164.172::INSTR')
mso.timeout = 5000  # Set timeout to 5 seconds
print(mso.query('*IDN?'))

AGILENT TECHNOLOGIES,MSO-X 2024A,MY53510379,02.41.2015102200



In [27]:
def print_scope_config(scope, channel="CHAN1"):
    print(f"Timebase Scale: {scope.query(':TIM:SCAL?').strip()} s/div")
    print(f"Sample Rate: {scope.query(':ACQ:SRAT?').strip()} Sa/s")
    #print(f"Memory Depth: {scope.query(':ACQ:MDEP?').strip()} samples")
    print(f"{channel} Voltage Scale: {scope.query(f':{channel}:SCAL?').strip()} V/div")
    print(f"{channel} Offset: {scope.query(f':{channel}:OFFS?').strip()} V")
    print(f"Trigger Source: {scope.query(':TRIG:SOUR?').strip()}")
    print(f"Trigger Level: {scope.query(':TRIG:LEV?').strip()} V")

In [28]:
print_scope_config(mso, channel="CHAN1")

Timebase Scale: +5.000E-06 s/div
Sample Rate: 1000000000 Sa/s
CHAN1 Voltage Scale: +1.00E+00 V/div
CHAN1 Offset: +1.15000E+00 V
Trigger Source: CHAN2
Trigger Level: +2.95000E+00 V
